In [1]:
using JLD2
using Statistics
using Revise
using Plots
using Laplacians
using Printf
using SparseArrays


In [2]:
function extractSlice(dic, slice)
    dicSlice = Dict()
    numepxr = length(dic["ne"])
    for (key, value) in dic
        if length(value) == numepxr
            dicSlice[key] = value[slice]
        else
            dicSlice[key] = value
        end
    end
    return dicSlice
end

extractSlice (generic function with 1 method)

In [3]:
function hasZeroDiag(M::SparseMatrixCSC{Tv, Ti}) where {Tv, Ti}
    for i in 1:size(M,1)
        if M[i,i] == zero(Tv)
            return true
        end
    end
    return false
end

function hasZeroDiag(name::String)
    M = downloadSS(name)
    return hasZeroDiag(M)
end

hasZeroDiag (generic function with 2 methods)

In [10]:
@load "suitesparse_all.jl.split02merge02.jld2"
dic1 = copy(dic);

In [11]:

sizeSlice = findall(dic1["ne"] .> 1000)
dic1 = extractSlice(dic1, sizeSlice);

In [12]:
include("../julia-files/downloadSS.jl")

downloadSS (generic function with 1 method)

In [13]:
zeroDiagSlice = findall(hasZeroDiag, dic1["testName"]);
noZeroDiagSlice = setdiff(1:length(sizeSlice), zeroDiagSlice);

In [14]:
dic1 = extractSlice(dic1, noZeroDiagSlice);

In [15]:
for name in dic1["testName"]
    println("- ", name)
end

- MaxPlanck/shallow_water2
- GHS_psdef/torsion1
- JGD_BIBD/bibd_81_2
- Norris/fv3
- GHS_psdef/obstclae
- HB/gr_30_30
- HB/bcsstm23
- HB/bcsstm24
- HB/bcsstm21
- Norris/fv1
- HB/bcsstm08
- HB/bcsstm09
- Boeing/bcsstm39
- HB/bcsstm26
- HB/bcsstm25
- Norris/fv2
- Oberwolfach/t3dl_e
- Gaertner/nopoly
- Andrews/Andrews
- GHS_psdef/jnlbrng1
- Oberwolfach/t2dal_e
- MaxPlanck/shallow_water1
- HB/nos6
- GHS_psdef/apache1
- HB/bcsstm11
- McRae/ecology1
- HB/nos7
- McRae/ecology2


In [16]:
function nanErr!(d)
    for (key, value) in d
        if occursin("err", key)
            for i in 1:length(value)
                if isnan(value[i])
                    d[key][i] = Inf
                end
            end
        end
    end
end

nanErr! (generic function with 1 method)

In [17]:
nanErr!(dic1);

In [18]:
using Printf

function sd(d,x)
    #scale = exp10(ceil(log10(abs(x)))-d)
    #return round(x/scale)*scale
    return round(x, sigdigits=3)
end        
ff(s) = (@sprintf "%.4g" sd(3,s))
nf(s) = (@sprintf "%d" sd(3,s))
ffK(s) = (@sprintf "%.4g" sd(3,s)/1000)
nfK(s) =  round(Int64, sd(3,s)/1000)
ffe8(s) = (@sprintf "%.3g" sd(3,s)*1e8)
ffe6(s) = (@sprintf "%.3g" sd(3,s)*1e6)
ffmicro(s) = (@sprintf "%.4g" sd(3,s)*1e6)

ffmicro (generic function with 1 method)

In [19]:
function labelTvn(tvn, err)
    if err <= 1e-8
        return ffe6(tvn)
    elseif err < 1e-4
        return string(ffe6(tvn), "\\tnote{*}")
    elseif err < 1
        return string(ffe6(tvn), "\\tnote{**}")
    else
        return "Inf"
    end
end

function printMedian(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = median(d[string(solver, "_tot")] ./ d["ne"] )
        err = median(d[string(solver, "_err")])
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

function print75(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = quantile(d[string(solver, "_tot")] ./ d["ne"], 0.75)
        err = quantile(d[string(solver, "_err")], 0.75)
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

function printMax(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = maximum(d[string(solver, "_tot")] ./ d["ne"])
        err = maximum(d[string(solver, "_err")])
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

printMax (generic function with 1 method)

In [20]:
function printSolverCell(d, solver) 
    return string(printMedian(d, solver), "&", print75(d, solver), "&", printMax(d, solver))
end

function printOneLine(d, solvers)
    allCells = [printSolverCell(d, solver) for solver in solvers]
    println(string(join(allCells, "&"), "\\\\"))
end

printOneLine (generic function with 1 method)

In [21]:
solvers=["ac", "ac-s2m2", "cmg2"]
printOneLine(dic1, solvers)

0.434&0.503&0.994&0.553&0.698&1.4&24.3&112&398\\


In [22]:
solvers = ["hypre", "petsc_hypre", "icc2"]
printOneLine(dic1, solvers)

0.0586&0.345&Inf&1&1.83&27.1&22.5&98.1&626\\
